# 4단계: 모델 빌딩 CI/CD 파이프라인 추가

<div class="alert alert-warning"> 이 노트북은 <code>SageMaker Distribution Image 3.6.1</code>을 사용하는 SageMaker Studio JupyterLab 인스턴스에서 SageMaker Python SDK 버전 <code>2.255.0</code>으로 마지막 테스트되었습니다</div>

이 단계에서는 [Amazon SageMaker Projects](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects.html)를 사용하여 모델 빌딩을 위한 자동화된 CI/CD 파이프라인을 생성합니다.

**아이디어에서 프로덕션까지 6단계:**
||||
|---|---|---|
|1. |노트북에서 실험 ||
|2. |SageMaker AI 처리 작업 및 SageMaker SDK로 확장 ||
|3. |ML 파이프라인, 모델 레지스트리, 피처 스토어로 운영화 ||
|4. |모델 빌딩 CI/CD 파이프라인 추가 |**<<<< 현재 위치**|
|5. |모델 배포 파이프라인 추가 ||
|6. |모델 및 데이터 모니터링 추가 ||

[SageMaker AI 제공 프로젝트 템플릿](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects-templates-sm.html#sagemaker-projects-templates-code-commit)을 사용하여 [AWS CodePipeline](https://aws.amazon.com/codepipeline/)과 GitHub 코드 저장소로 CI/CD 워크플로 자동화를 프로비저닝할 예정입니다.

SageMaker 프로젝트 템플릿은 다음과 같은 코드 저장소, 워크플로 자동화 도구, 파이프라인 단계 선택을 제공합니다:
- **코드 저장소**: GitHub 및 Bitbucket과 같은 타사 Git 저장소 지원
- **CI/CD 워크플로 자동화**: AWS CodePipeline 또는 Jenkins
- **파이프라인 단계**: 모델 빌딩, 훈련, 배포

<div class="alert alert-info"> 이 노트북에서는 JupyterLab에서 <code>Python 3</code> 커널을 사용하고 있는지 확인하세요.</div>

In [ ]:
import boto3
import sagemaker 
import json
from time import gmtime, strftime, sleep
from IPython.display import HTML

In [ ]:
%store -r 

%store

try:
    initialized
except NameError:
    print("+++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN 00-start-here notebook   ")
    print("+++++++++++++++++++++++++++++++++++++++++++++++++")

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
sm = boto3.client("sagemaker")

## MLOps 프로젝트 생성

### 준비

<div class="alert alert-info"><b>전제 조건:</b> 아래 설명된 대로 AWS 계정에서 GitHub 사용자 또는 조직으로의 AWS CodeStar 연결을 설정해야 합니다.<br/><b>이 AWS CodeStar 연결에 키 <code>sagemaker</code>와 값 <code>true</code>를 가진 태그를 추가하세요.</b></div>

자세한 지침은 SageMaker 개발자 가이드의 [타사 Git 저장소를 사용한 SageMaker AI MLOps 프로젝트 연습](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects-walkthrough-3rdgit.html)을 참조하세요.

<div class="alert alert-info">GitHub 계정에 두 개의 새로운 비어있는 프라이빗 저장소를 생성해야 합니다. 하나는 모델 빌드 파이프라인용이고 다른 하나는 모델 배포 파이프라인용입니다. 예를 들어, 저장소 이름을 <code>model-build</code>와 <code>model-deploy</code>로 지정할 수 있습니다</div>

#### GitHub 연결 설정

[AWS CodeConnection 연결](https://docs.aws.amazon.com/dtconsole/latest/userguide/welcome-connections.html)을 통해 접근하고 연결할 수 있는 GitHub 개인 계정이 필요합니다.

다음 지침에 따라 GitHub에 연결하세요 (자세한 내용은 [문서](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects-walkthrough-3rdgit.html#sagemaker-proejcts-walkthrough-connect-3rdgit)를 참조하세요).

먼저 GitHub 개인 계정에 두 개의 빈 저장소를 생성합니다. 이 두 저장소는 모델 빌드 워크플로우와 모델 배포 워크플로우의 시드 코드로 채워집니다. [GitHub.com](https://github.com/)으로 이동하여 사용자 드롭다운 메뉴에서
Your repositories로 이동합니다.
model-build 저장소를 생성합니다.

![](img/github-model-build.png)

model-deploy 저장소에 대해서도 동일한 과정을 반복합니다.

![](img/github-model-deploy.png)

이제 코드 변경 시 워크플로우 트리거를 활성화하기 위해 [Connections](https://docs.aws.amazon.com/dtconsole/latest/userguide/welcome-connections.html)(AWS 내 Developer Tools의 일부)를 통해 개인 GitHub 계정을 AWS 계정
에 연결해야 합니다. [Connections](https://console.aws.amazon.com/codesuite/settings/connections)로 이동하여 새 연결을 생성합니다.

![](img/crete-connection.png)

이제 GitHub를 공급자로 선택하고, 예를 들어 mlops-connection과 같은 이름을 지정한 후, 키는 sagemaker이고 값은 true인 새 태그를 추가합니다.

![](img/github-connection.png)

GitHub 계정에 인증하라는 메시지가 표시됩니다. 모범 사례로, 새 앱을 설치하여 방금 생성한 두 저장소(model-build 및 model-deploy)에만 권한을 제한할 수 있도록 하는 것을 권장합니다.

![](img/install-github-app.png)
<img src="img/connection-permissions.png" width="500" height="800" alt="설명">

연결을 생성하고 아래 셀에 ARN을 복사하여 붙여넣습니다.
참고: 이 전체 과정은 빈 저장소를 생성하는 한 지원되는 모든 공급자에 대해서도 수행할 수 있습니다.

In [ ]:
# set this variable to the ARN of your code connection your created
code_connection_arn = <SET TO THE ARN OF THE CREATED CODE CONNECTION>

이 노트북에서 프로그래밍 방식으로 프로젝트를 생성할 수 있습니다 - **옵션 1** 또는 Studio UI에서 - **옵션 2**.

옵션 1은 UX에 대한 종속성이 없으므로 권장됩니다</br>
옵션 2는 [**프로젝트 생성** UI 플로우](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects-create.html)를 시연하기 위해 제공됩니다.

### 옵션 1: 프로그래밍 방식으로 프로젝트 생성 - 권장
이 섹션에서는 `boto3`를 사용하여 SageMaker API를 통해 MLOps 프로젝트를 생성합니다.

In [ ]:
sc = boto3.client("servicecatalog")

sc_provider_name = "Amazon SageMaker"
sc_product_name = "MLOps template for model building, training, and deployment with third-party Git repositories using CodePipeline"

In [ ]:
# find a Service Catalog product with the specific SageMaker project template
p_ids = [p['ProductId'] for p in sc.search_products(
    Filters={
        'FullTextSearch': [sc_product_name]
    },
)['ProductViewSummaries'] if p["Name"]==sc_product_name]

In [ ]:
p_ids

In [ ]:
# If you get any exception from this code, go to the Option 2 and create a project in Studio UI
if not len(p_ids):
    raise Exception("No Amazon SageMaker ML Ops products found!")
elif len(p_ids) > 1:
    raise Exception("Too many matching Amazon SageMaker ML Ops products found!")
else:
    product_id = p_ids[0]
    print(f"ML Ops product id: {product_id}")

In [ ]:
# output what this project is about
sc.describe_product(Id=product_id)['ProductViewSummary']['ShortDescription']

In [ ]:
# get the latest template version
provisioning_artifact_id = sorted(
    [i for i in sc.list_provisioning_artifacts(
        ProductId=product_id
    )['ProvisioningArtifactDetails'] if i['Guidance']=='DEFAULT'],
    key=lambda d: d['Name'], reverse=True)[0]['Id']

In [ ]:
provisioning_artifact_id

In [ ]:
sc.describe_provisioning_artifact(ProductId=product_id, ProvisioningArtifactId=provisioning_artifact_id)

In [ ]:
# set unique project name
project_name = f"mlops-{strftime('%m-%d-%H-%M-%S', gmtime())}"

프로젝트 템플릿의 매개변수를 설정합니다. `model_build_code_repository_full_name`과 `model_deploy_code_repository_full_name` 변수를 GitHub 저장소의 전체 이름으로 설정하세요. 이들은 새로운 빈 저장소여야 합니다.

In [ ]:
# Branch name for the Model Building and Training Code Repository
model_build_code_repository_branch = 'main'
# Full repository name of the Model Building and Training Code Repository, which would be username/reponame or organizationname/reponame
model_build_code_repository_full_name = <ENTER YOUR FULL GITHUB REPO FOR MODEL BUILD NAME HERE> # e.g. username/reponame

# Branch name for the Model Deployment Code Repository
model_deploy_code_repository_branch = 'main'
# Full repository name of the Model Deployment Code Repository, which would be username/reponame or organizationname/reponame
model_deploy_code_repository_full_name = <ENTER YOUR FULL GITHUB REPO FOR MODEL DEPLOY NAME HERE> # e.g. username/reponame

In [ ]:
# set project parameters
project_parameters = [
    {
        'Key': 'ModelBuildCodeRepositoryBranch',
        'Value': model_build_code_repository_branch,
    },
    {
        'Key': 'ModelBuildCodeRepositoryFullname',
        'Value': model_build_code_repository_full_name,
    },
    {
        'Key': 'ModelDeployCodeRepositoryBranch',
        'Value': model_deploy_code_repository_branch,
    },
    {
        'Key': 'ModelDeployCodeRepositoryFullname',
        'Value': model_deploy_code_repository_full_name,
    },
        {
        'Key': 'CodeConnectionArn',
        'Value': code_connection_arn,
    },
]

마지막으로, 서비스 카탈로그 제품 템플릿에서 SageMaker 프로젝트를 생성합니다:

In [ ]:
print(f'''Creating a {project_name} using {sc_product_name} with the following parameters:
{json.dumps(project_parameters, indent=2)}
''')

In [ ]:
# create SageMaker project
r = sm.create_project(
    ProjectName=project_name,
    ProjectDescription="Model build and deploy project",
    ServiceCatalogProvisioningDetails={
        'ProductId': product_id,
        'ProvisioningArtifactId': provisioning_artifact_id,
        'ProvisioningParameters': project_parameters
    },
)

print(r)
project_id = r["ProjectId"]

<div class="alert alert-info">다음 셀을 실행하여 프로젝트 생성이 완료될 때까지 기다리세요</div>

In [ ]:
# Project creation takes about 3-5 min
while sm.describe_project(ProjectName=project_name)['ProjectStatus'] not in ['CreateCompleted', 'CreateFailed']:
    print("Waiting for project creation completion")
    sleep(10)
    
print(f"MLOps project {project_name} creation completed")

In [ ]:
assert sm.describe_project(ProjectName=project_name)['ProjectStatus'] == 'CreateCompleted', 'Project status must be CreateCompleted!'

### 옵션 1 종료: 프로그래밍 방식으로 프로젝트 생성
이제 SageMaker 환경에서 프로젝트 템플릿을 프로비저닝했습니다. **MLOps 프로젝트 구성** 섹션으로 이동하세요.

---

### 옵션 2: Studio UI에서 프로젝트 생성
<div class="alert alert-info"><b>프로그래밍 방식으로 이미 프로젝트를 생성했다면 이 섹션을 건너뛰세요.</b></div>

<div class="alert alert-info">MLOps 프로젝트를 생성하기 전에 <b>준비</b> 단계를 완료해야 합니다.</div>

개발자 가이드의 [**2단계: 프로젝트 생성** 지침](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects-walkthrough-3rdgit.html#sagemaker-proejcts-walkthrough-create-3rdgit)에 따라 MLOps CI/CD 프로젝트를 생성하세요.

템플릿으로는 **CodePipeline을 사용한 타사 Git 저장소와 함께하는 모델 빌드, 훈련 및 배포**를 선택하세요:

![](img/mlops-project-name.png)

프로젝트가 생성될 때까지 기다리세요.

### 프로젝트 생성 문제 해결

#### 오류 메시지
❗ 다음과 유사한 오류 메시지가 표시되는 경우:

Your project couldn't be created
Studio encountered an error when creating your project. Try recreating the project again.

CodeBuild is not authorized to perform: sts:AssumeRole on arn:aws:iam::XXXX:role/service-role/AmazonSageMakerServiceCatalogProductsCodeBuildRole (Service: AWSCodeBuild; Status Code: 400; Error Code:
InvalidInputException; Request ID: 4cf59a54-0c59-476a-a970-0ac656db4402; Proxy: null)

[프로젝트 사용에 필요한 SageMaker Studio 권한](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects-studio-updates.html)의 5-6단계를 참조하세요. **프로젝트** 아래 **앱** 카드에 나열된 모든 필수 프로젝트 역할이 있는지 확인하세요.

또는 제공된 CloudFormation 템플릿 [`cfn-templates/sagemaker-project-templates-roles.yaml`](cfn-templates/sagemaker-project-templates-roles.yaml)을 사용하여 필요한 역할을 생성할 수 있습니다.
해당 권한이 있는 명령줄 터미널에서 저장소 복제 디렉터리에서 실행하세요:

```sh
aws cloudformation deploy \
   --template-file cfn-templates/sagemaker-project-templates-roles.yaml \
   --stack-name sagemaker-project-template-roles \
   --capabilities CAPABILITY_IAM CAPABILITY_NAMED_IAM \
   --parameter-overrides \
   CreateCloudFormationRole=YES \
   CreateCodeBuildRole=YES \
   CreateCodePipelineRole=YES \
   CreateEventsRole=YES \
   CreateProductsExecutionRole=YES
```

### 옵션 2 종료: Studio UI에서 프로젝트 생성
이제 프로젝트가 생성되었으므로 **MLOps 프로젝트 구성** 섹션으로 이동하세요.

---

## MLOps 프로젝트 구성
프로비저닝된 MLOps 프로젝트는 두 개의 CodePipeline 파이프라인을 구현합니다. 하나는 모델 빌드용이고 다른 하나는 모델 배포용입니다. 이 프로젝트는 또한 지정한 두 개의 GitHub 저장소를 시드 코드로 채웁니다. 이 노트북은 모델 빌드 파이프라인을 설명하고 구성합니다. 다음 노트북 `05-deploy`는 두 번째 부분인 모델 배포 파이프라인을 다룹니다.

### 모델 빌드 파이프라인

프로젝트는 생성되자마자 기본 모델 빌드 파이프라인을 자동으로 프로비저닝하고 실행합니다. 이 파이프라인은 사용자 고유의 커스텀 파이프라인을 위한 프로젝트 내 샘플 플레이스홀더입니다. 지금은 기본 파이프라인을 무시하세요.
프로젝트 템플릿은 AWS 계정에 다음 리소스를 배포합니다:

![](img/mlops-model-build-train.png)

주요 구성 요소는 다음과 같습니다:
1. 프로젝트 템플릿은 SageMaker Projects와 AWS Service Catalog 포트폴리오를 통해 제공됩니다
2. 두 단계로 구성된 CodePipeline 파이프라인 - GitHub 저장소에서 소스 코드를 다운로드하는 `Source`와 SageMaker 파이프라인을 생성하고 실행하는 `Build`
3. 모델 빌드, 훈련 및 등록 워크플로우가 포함된 기본 SageMaker 파이프라인
4. 제공된 기본 버전의 시드 코드가 있는 GitHub의 시드 코드 저장소

이 프로젝트는 사전 정의된 템플릿에서 자동화된 CI/CD 파이프라인을 구현하는 데 필요한 모든 코드와 인프라를 포함합니다.
프로젝트를 파이프라인과 함께 사용하기 시작하려면 다음 단계를 완료해야 합니다:
1. 프로젝트 GitHub 저장소를 노트북 로컬 스토리지에 복제
2. 3단계 노트북에서 구현된 실제 파이프라인 구성 코드로 ML 파이프라인 템플릿 샘플 코드 교체
3. 올바른 Python 모듈 이름을 참조하고 프로젝트 매개변수를 설정하도록 `codebuild-buildspec.yml` 파일 수정

다음 섹션에서 이러한 단계를 안내합니다. 자세한 지침과 실습 예제는 개발 가이드 [SageMaker MLOps 프로젝트 연습](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects-walkthrough-3rdgit.html)을 참조하세요.

옵션 1 `boto3`를 사용하여 MLOps 프로젝트를 생성한 경우 `project_name`과 `project_id`가 자동으로 설정됩니다. 다음 코드 셀을 실행하여 값을 출력할 수 있습니다. UI 지침에 따라 프로젝트를 생성한 경우 `project_name`을 수동으로 설정해야 합니다.

In [ ]:
try:
    print(project_name)
    print(project_id)
    print(model_build_code_repository_full_name)
    print(code_connection_arn)
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("Set the code_connection_arn, project_name, and repository full name in the following code cell")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

In [ ]:
# project_name = "<SET TO THE NAME OF THE CREATED PROJECT>" # Keep commented out if you used option 1 to create a project
# model_build_code_repository_full_name = "<SET TO THE FULL NAME OF MODEL BUILD REPO>" # Keep commented out if you used option 1 to create a project

r = sm.describe_project(ProjectName=project_name)
project_id = r['ProjectId']
project_arn = r['ProjectArn']
repository_name = model_build_code_repository_full_name.split('/')[1]
git_folder = project_name
project_folder = f'sagemaker-{project_id}-modelbuild'
project_path = f'{git_folder}/{project_folder}'

%store project_name
%store project_arn
%store project_id
    
print(f"Project path: {project_path}")

### Studio UI에서 프로젝트 탐색

다음 코드 셀에서 구성된 링크를 클릭하세요. Studio UI에서 프로젝트와 프로젝트 구성 요소를 탐색하세요.

In [ ]:
# Show the project link
display(
    HTML('<b>See <a target="top" href="https://studio-{}.studio.{}.sagemaker.aws/projects/{}/">the project</a> in the Studio UI</b>'.format(
            domain_id, region, project_name))
)

### 1. JupyterLab 파일 시스템에 프로젝트 시드 코드 복제
JupyterLab 터미널과 GitHub CLI를 사용하여 GitHub 저장소에서 프로젝트 코드를 복제해야 합니다.

1. **File** > **New** > **Terminal**을 통해 새 터미널 창을 엽니다
2. 다음 셀의 출력을 복사하여 터미널 명령줄에 붙여넣습니다
3. `codeconnections` 자격 증명 도우미와 함께 `git clone`을 사용하여 모델 빌드 저장소를 복제합니다.

In [ ]:
def copy_output(text):
    return HTML(f'''
        <div style="position: relative;">
            <pre>{text}</pre>
            <button onclick="navigator.clipboard.writeText(`{text}`)"
                    style="position: absolute; top: 5px; right: 5px;">
                Copy
            </button>
        </div>
    ''')
    
cmd = f'''
git config --global credential.UseHttpPath true
git config --global credential.helper 'cache --timeout=720000'
git config --global credential.helper '!aws codecommit credential-helper $@'

mkdir -p $HOME/{project_path}

git clone https://codeconnections.{region}.amazonaws.com/git-http/{code_connection_arn.split(':')[4]}/{region}/{code_connection_arn.split(':')[-1].split('/')[-1]}/{model_build_code_repository_full_name}.git  $HOME/{project_path}
'''

copy_output(cmd)

### 2. 파이프라인 구성 코드 교체

시드 코드가 포함된 프로젝트를 사용자 정의하려면 다음 단계가 필요합니다. 다음 코드 셀은 필요한 모든 단계를 실행하므로 수동으로 수행할 필요가 없습니다. 다음 텍스트는 정보 제공 목적입니다.

- 시드 소스 코드는 `<project_name>/sagemaker-<project-id>-modelbuild` 폴더에 있습니다.
- 원본 파일 `codebuild-buildspec.yml`은 `codebuild-buildspec-original.yml`로 이름이 변경됩니다.
- 파이프라인 코드가 포함된 MLOps 프로젝트의 코드 저장소 폴더는 `abalone` 폴더에서 `fromideatoprod`로 이름이 변경됩니다.
- 템플릿 파이프라인이 포함된 원본 파일 `pipeline.py`는 `pipeline-original.py`로 이름이 변경됩니다.
- `pipeline_steps` Python 모듈을 MLOps 프로젝트의 코드 저장소 폴더 내 `pipelines` 폴더에 복사합니다.
- 노트북 3에서 생성된 `requirements.txt`를 MLOps 프로젝트의 코드 저장소 폴더 내 `pipelines` 폴더에 복사합니다.
- 노트북 3의 SageMaker Python SDK 기본 구성 파일 `config.yaml`을 MLOps 프로젝트의 코드 저장소 폴더 내 `pipelines` 폴더에 복사합니다.

In [ ]:
# see the workshop folder name
!pwd

In [ ]:
# if you local path for the workshop folder is different, set the correct absolute path to the variable workshop_folder
workshop_folder = "amazon-sagemaker-from-idea-to-production"

In [ ]:
!mkdir -p ~/{workshop_folder}/pipelines
!mv ~/{project_path}/codebuild-buildspec.yml ~/{project_path}/codebuild-buildspec-original.yml
!mv ~/{project_path}/setup.py ~/{project_path}/setup-original.py
!mv ~/{project_path}/pipelines/abalone ~/{project_path}/pipelines/fromideatoprod
!mv ~/{project_path}/pipelines/fromideatoprod/pipeline.py ~/{project_path}/pipelines/fromideatoprod/pipeline-original.py
!cp ~/{workshop_folder}/pipeline_steps/* ~/{project_path}/pipelines/
!cp ~/{workshop_folder}/pipeline_steps/* ~/{workshop_folder}/pipelines/
!cp ~/{workshop_folder}/requirements.txt ~/{project_path}
!cp ~/{workshop_folder}/config.yaml ~/{project_path}

다음 셀을 실행하여 파이프라인 구성 코드를 `pipeline.py` 파일에 작성합니다. 3단계 노트북의 코드를 `get_pipeline()` 함수로 재사용합니다.

<div style="border: 4px solid coral; text-align: center; margin: auto;">
    <p style=" text-align: center; margin: auto;">파이프라인 구성 코드는 S3 원시 입력 데이터셋과 Feature Store의 피처 세트 모두에서 작동하며, 파이프라인에 해당 입력 매개변수를 전달해야 합니다.
    </p>
</div>

In [ ]:
%%writefile pipeline.py

import pandas as pd
import json
import boto3
import pathlib
import io
import os
import sagemaker
import mlflow
from time import gmtime, strftime, sleep
from sagemaker.deserializers import CSVDeserializer
from sagemaker.serializers import CSVSerializer

from sagemaker.workflow.execution_variables import ExecutionVariables
from sagemaker.workflow.pipeline_context import PipelineSession
from sagemaker.xgboost.estimator import XGBoost
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import (
    ProcessingInput, 
    ProcessingOutput, 
    ScriptProcessor
)
from sagemaker.inputs import TrainingInput

from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.steps import (
    ProcessingStep, 
    TrainingStep, 
    CreateModelStep,
    CacheConfig
)
from sagemaker.workflow.check_job_config import CheckJobConfig
from sagemaker.workflow.parameters import (
    ParameterInteger, 
    ParameterFloat, 
    ParameterString, 
    ParameterBoolean
)
from sagemaker.workflow.quality_check_step import (
    DataQualityCheckConfig,
    ModelQualityCheckConfig,
    QualityCheckStep,
)
from sagemaker.workflow.clarify_check_step import (
    ModelBiasCheckConfig, 
    ClarifyCheckStep, 
    ModelExplainabilityCheckConfig
)
from sagemaker import Model
from sagemaker.inputs import CreateModelInput
from sagemaker.workflow.model_step import ModelStep
from sagemaker.workflow.fail_step import FailStep
from sagemaker.workflow.conditions import (
    ConditionGreaterThan,
    ConditionGreaterThanOrEqualTo
)
from sagemaker.workflow.parallelism_config import ParallelismConfiguration
from sagemaker.workflow.properties import PropertyFile
from sagemaker.workflow.condition_step import ConditionStep
from sagemaker.workflow.functions import (
    Join,
    JsonGet
)
from sagemaker.workflow.lambda_step import (
    LambdaStep,
    LambdaOutput,
    LambdaOutputTypeEnum,
)
from sagemaker.lambda_helper import Lambda

from sagemaker.model_metrics import (
    MetricsSource, 
    ModelMetrics, 
    FileSource
)
from sagemaker.drift_check_baselines import DriftCheckBaselines
from sagemaker.workflow.pipeline_definition_config import PipelineDefinitionConfig 
from sagemaker.image_uris import retrieve
from sagemaker.workflow.function_step import step
from sagemaker.workflow.step_outputs import get_step
from sagemaker.model_monitor import DatasetFormat, model_monitoring

from pipelines.preprocess import preprocess
from pipelines.evaluate import evaluate
from pipelines.register import register
from pipelines.extract import prepare_datasets

def get_sagemaker_client(region):
     return boto3.Session(region_name=region).client("sagemaker")

def get_pipeline_session(region, bucket_name):
    """Gets the pipeline session based on the region.

    Args:
        region: the aws region to start the session
        bucket_name: the bucket to use for storing the artifacts

    Returns:
        PipelineSession instance
    """

    boto_session = boto3.Session(region_name=region)
    sagemaker_client = boto_session.client("sagemaker")

    return PipelineSession(
        boto_session=boto_session,
        sagemaker_client=sagemaker_client,
        default_bucket=bucket_name,
    )

def get_pipeline_custom_tags(new_tags, region, sagemaker_project_name=None):
    try:
        print(f"Getting project tags for {sagemaker_project_name}")
        
        sm_client = get_sagemaker_client(region)
        
        project_arn = sm_client.describe_project(ProjectName=sagemaker_project_name)['ProjectArn']
        project_tags = sm_client.list_tags(ResourceArn=project_arn)['Tags']

        print(f"Project tags: {project_tags}")
        
        for project_tag in project_tags:
            new_tags.append(project_tag)
            
    except Exception as e:
        print(f"Error getting project tags: {e}")
        
    return new_tags
    
def get_pipeline(
    region,
    sagemaker_project_id=None,
    sagemaker_project_name=None,
    role=None,
    bucket_name=None,
    bucket_prefix="from-idea-to-prod/xgboost",
    input_s3_url=None,
    feature_group_name=None,
    model_package_group_name="from-idea-to-prod-model-group",
    pipeline_name_prefix="from-idea-to-prod-pipeline",
    process_instance_type="ml.m5.large",
    train_instance_type="ml.m5.xlarge",
    test_score_threshold=0.70,
    tracking_server_arn=None,
):
    """Gets a SageMaker ML Pipeline instance.
    
    Returns:
        an instance of a pipeline
    """
    if feature_group_name is None and input_s3_url is None:
        print("One of feature_group_name or input_s3_url must be provided. Exiting...")
        return None

    session = get_pipeline_session(region, bucket_name)
    sm = session.sagemaker_client
    
    if role is None:
        role = sagemaker.session.get_execution_role(session)

    print(f"sagemaker version: {sagemaker.__version__}")
    print(f"Execution role: {role}")
    print(f"Input S3 URL: {input_s3_url}")
    print(f"Feature group: {feature_group_name}")
    print(f"Model package group: {model_package_group_name}")
    print(f"Pipeline name prefix: {pipeline_name_prefix}")
    print(f"Tracking server ARN: {tracking_server_arn}")
    
    pipeline_name = f"{pipeline_name_prefix}-{sagemaker_project_id}"
    experiment_name = pipeline_name

    output_s3_prefix = f"s3://{bucket_name}/{bucket_prefix}"
    # Set the output S3 url for model artifact
    output_s3_url = f"{output_s3_prefix}/output"
    # Set the output S3 url for feature store query results
    output_query_location = f'{output_s3_prefix}/offline-store/query_results'
    
    # Set the output S3 urls for processed data
    train_s3_url = f"{output_s3_prefix}/train"
    validation_s3_url = f"{output_s3_prefix}/validation"
    test_s3_url = f"{output_s3_prefix}/test"
    evaluation_s3_url = f"{output_s3_prefix}/evaluation"
    
    baseline_s3_url = f"{output_s3_prefix}/baseline"
    prediction_baseline_s3_url = f"{output_s3_prefix}/prediction_baseline"
    
    xgboost_image_uri = sagemaker.image_uris.retrieve(
            "xgboost", 
            region=region,
            version="1.5-1"
    )

    # If no tracking server ARN, try to find an active MLflow server
    if tracking_server_arn is None:
        r = sm.list_mlflow_tracking_servers(
            TrackingServerStatus='Created',
        )['TrackingServerSummaries']
    
        if len(r) < 1:
            print("You don't have any running MLflow servers. Exiting...")
            return None
        else:
            tracking_server_arn = r[0]['TrackingServerArn']
            print(f"Use the tracking server ARN:{tracking_server_arn}")
        
    # Parameters for pipeline execution
    
    # Set processing instance type
    process_instance_type_param = ParameterString(
        name="ProcessingInstanceType",
        default_value=process_instance_type,
    )

    # Set training instance type
    train_instance_type_param = ParameterString(
        name="TrainingInstanceType",
        default_value=train_instance_type,
    )

    # Set model approval param
    model_approval_status_param = ParameterString(
        name="ModelApprovalStatus",
        default_value="PendingManualApproval"
    )

    # Minimal threshold for model performance on the test dataset
    test_score_threshold_param = ParameterFloat(
        name="TestScoreThreshold", 
        default_value=test_score_threshold
    )

    # S3 url for the input dataset
    input_s3_url_param = ParameterString(
        name="InputDataUrl",
        default_value=input_s3_url if input_s3_url else "None",
    )

    # Feature group name for the input featureset
    feature_group_name_param = ParameterString(
        name="FeatureGroupName",
        default_value=feature_group_name if feature_group_name else "None",
    )
    
    # Model package group name
    model_package_group_name_param = ParameterString(
        name="ModelPackageGroupName",
        default_value=model_package_group_name,
    )

    # MLflow tracking server ARN
    tracking_server_arn_param = ParameterString(
        name="TrackingServerARN",
        default_value=tracking_server_arn,
    )
    
    # Define step cache config
    cache_config = CacheConfig(
        enable_caching=True,
        expire_after="P30d" # 30-day
    )

    # Construct the pipeline
    
    # Get datasets
    step_get_datasets = step(
            preprocess, 
            role=role,
            instance_type=process_instance_type_param,
            name=f"preprocess",
            keep_alive_period_in_seconds=3600,
    )(
        input_data_s3_path=input_s3_url_param,
        output_s3_prefix=output_s3_prefix,
        tracking_server_arn=tracking_server_arn_param,
        experiment_name=experiment_name,
        pipeline_run_name=ExecutionVariables.PIPELINE_EXECUTION_ID,
    ) if input_s3_url else step(
        prepare_datasets, 
        role=role,
        instance_type=process_instance_type_param,
        name=f"extract-featureset",
        keep_alive_period_in_seconds=3600,
    )(
        feature_group_name=feature_group_name_param,
        output_s3_prefix=output_s3_prefix,
        query_output_s3_path=output_query_location,
        tracking_server_arn=tracking_server_arn_param,
        experiment_name=experiment_name,
        pipeline_run_name=ExecutionVariables.PIPELINE_EXECUTION_ID,
    )
    
    # Instantiate an XGBoost estimator object
    estimator = sagemaker.estimator.Estimator(
        image_uri=xgboost_image_uri,
        role=role, 
        instance_type=train_instance_type_param,
        instance_count=1,
        output_path=output_s3_url,
        sagemaker_session=session,
        base_job_name=f"{pipeline_name}-train"
    )
    
    # Define algorithm hyperparameters
    estimator.set_hyperparameters(
        num_round=100, # the number of rounds to run the training
        max_depth=3, # maximum depth of a tree
        eta=0.5, # step size shrinkage used in updates to prevent overfitting
        alpha=2.5, # L1 regularization term on weights
        objective="binary:logistic",
        eval_metric="auc", # evaluation metrics for validation data
        subsample=0.8, # subsample ratio of the training instance
        colsample_bytree=0.8, # subsample ratio of columns when constructing each tree
        min_child_weight=3, # minimum sum of instance weight (hessian) needed in a child
        early_stopping_rounds=10, # the model trains until the validation score stops improving
        verbosity=1, # verbosity of printing messages
    )
    
    # train step
    step_train = TrainingStep(
        name=f"train",
        step_args=estimator.fit(
            {
                "train": TrainingInput(
                    step_get_datasets['train_data'],
                    content_type="text/csv",
                ),
                "validation": TrainingInput(
                    step_get_datasets['validation_data'],
                    content_type="text/csv",
                ),
            }
        ),
        cache_config=cache_config,
    )   
    
    # Evaluation step
    step_evaluate = step(
        evaluate,
        role=role,
        instance_type=process_instance_type_param,
        name=f"evaluate",
        keep_alive_period_in_seconds=3600,
    )(
        test_x_data_s3_path=step_get_datasets['test_x_data'],
        test_y_data_s3_path=step_get_datasets['test_y_data'],
        model_s3_path=step_train.properties.ModelArtifacts.S3ModelArtifacts,
        output_s3_prefix=output_s3_prefix,
        tracking_server_arn=tracking_server_arn_param,
        experiment_name=step_get_datasets['experiment_name'],
        pipeline_run_id=step_get_datasets['pipeline_run_id'],
    )

    # register model step
    step_register = step(
        register,
        role=role,
        instance_type=process_instance_type_param,
        name=f"register",
        keep_alive_period_in_seconds=3600,
    )(
        training_job_name=step_train.properties.TrainingJobName,
        model_package_group_name=model_package_group_name_param,
        model_approval_status=model_approval_status_param,
        evaluation_result=step_evaluate['evaluation_result'],
        output_s3_prefix=output_s3_url,
        tracking_server_arn=tracking_server_arn_param,
        experiment_name=step_get_datasets['experiment_name'],
        pipeline_run_id=step_get_datasets['pipeline_run_id'],
    )

    # fail the pipeline execution step
    step_fail = FailStep(
        name=f"fail",
        error_message=Join(on=" ", values=["Execution failed due to AUC Score < ", test_score_threshold_param]),
    )
    
    # condition to check in the condition step
    condition_gte = ConditionGreaterThanOrEqualTo(
            left=step_evaluate['evaluation_result']['classification_metrics']['auc_score']['value'],  
            right=test_score_threshold_param,
    )
    
    # conditional register step
    step_conditional_register = ConditionStep(
        name=f"check-metrics",
        conditions=[condition_gte],
        if_steps=[step_register],
        else_steps=[step_fail],
    )   

    # Create a pipeline object
    pipeline = Pipeline(
        name=f"{pipeline_name}",
        parameters=[
            input_s3_url_param,
            feature_group_name_param,
            process_instance_type_param,
            train_instance_type_param,
            model_approval_status_param,
            test_score_threshold_param,
            model_package_group_name_param,
            tracking_server_arn_param,
        ],
        steps=[step_conditional_register],
        pipeline_definition_config=PipelineDefinitionConfig(use_custom_job_prefix=True)
    )
    
    return pipeline

워크샵 폴더에서 이 `pipeline.py` 파일을 프로젝트의 코드 저장소 폴더 내 `pipelines/fromideatoprod` 폴더로 복사합니다:

In [ ]:
!cp ~/{workshop_folder}/pipeline.py ~/{project_path}/pipelines/fromideatoprod/

원격으로 실행하기 전에 모든 것이 작동하는지 확인하기 위해 `get_pipeline` 함수를 로컬에서 테스트합니다.

In [ ]:
from pipeline import get_pipeline

In [ ]:
# If you created a feature store in the notebook 3, you can set the feature_group_name parameter instead of input_s3_url to take the data from the feature store
p = get_pipeline(
    region=region,
    sagemaker_project_id=project_id,
    sagemaker_project_name=project_name,
    role=sm_role,
    bucket_name=bucket_name,
    bucket_prefix=bucket_prefix,
    input_s3_url=input_s3_url,
    # feature_group_name=dataset_feature_group_name,
    model_package_group_name=model_package_group_name,
    pipeline_name_prefix=pipeline_name,
    process_instance_type="ml.m5.large",
    train_instance_type="ml.m5.xlarge",
    test_score_threshold=0.70,
    tracking_server_arn=mlflow_arn,
)

In [ ]:
p.definition()

In [ ]:
p.upsert(role_arn=sm_role)

Studio UI에서 생성된 파이프라인을 보려면 아래 코드 셀에서 구성된 링크를 클릭하세요:

In [ ]:
from IPython.display import HTML

# Show the pipeline link
display(
    HTML('<b>See <a target="top" href="https://studio-{}.studio.{}.sagemaker.aws/pipelines/{}/graph">the pipeline</a> in the Studio UI</b>'.format(
            domain_id, region, p.describe()['PipelineName']))
)

이 시점에서 파이프라인 구성 코드가 작동하고 파이프라인을 생성한다는 것을 로컬에서 테스트했습니다. Studio **Pipelines** 위젯에서 이 파이프라인을 볼 수 있습니다. 이제 CI/CD 파이프라인을 생성할 준비가 되었습니다.

#### 리소스 태그로 프로젝트 소유권 제어
프로젝트 소유 리소스는 비용 제어, 속성 기반 보안 제어 및 거버넌스를 위해 `sagemaker:project-name` 및 `sagemaker:project-id` 태그로 자동 태그됩니다.

기존 저장소, 파이프라인, 모델 레지스트리 그룹 또는 엔드포인트를 프로젝트에 연결해야 하는 경우, 프로젝트 이름과 ID가 포함된 이 두 태그를 리소스에 추가할 수 있습니다.

예를 들어, 다음 코드 셀은 이전 노트북에서 생성한 모델 패키지 그룹에 `sagemaker:*` 태그를 추가합니다. 결과적으로 모델 패키지 그룹이 이제 프로젝트 리소스로 표시됩니다.

In [ ]:
model_package_group_arn = sm.describe_model_package_group(ModelPackageGroupName=model_package_group_name).get("ModelPackageGroupArn")

if model_package_group_arn:
    print(f"Adding tags {project_arn.split('/')[-1]} and {project_id} for model package group {model_package_group_arn}")
    r = sm.add_tags(
        ResourceArn=model_package_group_arn,
        Tags=[
            {
                'Key': 'sagemaker:project-name',
                'Value': project_arn.split("/")[-1]
            },
            {
                'Key': 'sagemaker:project-id',
                'Value': project_id
            },
        ]
    )
    print(r)
else:
    print(f"The model package group {model_package_group_name} doesn't exist")
    
sm.list_tags(ResourceArn=model_package_group_arn)["Tags"]

### 3. Modify the build specification file
Now modify the `codebuild-buildspec.yml` file in the project folder to reflect the new name of the Python module with your pipeline and set other project-specific parameters.

방금 생성한 `get_pipeline` 함수의 매개변수에 해당하는 다음 매개변수를 파이프라인 생성 스크립트에 전달해야 합니다:
- `input_s3_url` - 입력 원시 데이터셋을 위한 S3 URL. 노트북 3에서 피처 그룹을 생성한 경우 대신 `feature_group_name` 매개변수를 사용할 수 있습니다.
- `feature_group_name` – 노트북 3에서 이 피처 그룹을 생성한 경우 이 매개변수를 사용할 수 있으며, 이 경우 `input_s3_url`을 제공할 필요가 없습니다.
- `model_package_group_name` – 훈련 후 모델을 등록할 모델 레지스트리 패키지
- `pipeline_name_prefix` – 파이프라인의 이름 접두사. 파이프라인 이름은 `<pipeline_name_prefix>-<project-id>`로 구성됩니다.
- `role` – 파이프라인 실행 역할
- `tracking_server_arn`- 파이프라인 실행 추적을 위한 MLflow 서버 ARN

다음 셀들은 이러한 매개변수의 값을 출력합니다:

In [ ]:
try:
    print(f"""
        INPUT-S3-URL: {input_s3_url}
        FEATURE-GROUP-NAME: {dataset_feature_group_name}
        MODEL-PACKAGE-GROUP-NAME: {project_name}-{project_id}
        PIPELINE-NAME-PREFIX: {pipeline_name}
        ROLE: {sm_role}
        TRACKING-SERVER-ARN: {mlflow_arn}
        """)
except NameError:
    print(f"""
        Dataset feature group name is not defined, use input_s3_url instead:
        ********************************************************************
        
        INPUT-S3-URL: {input_s3_url}
        MODEL-PACKAGE-GROUP-NAME: {project_name}-{project_id}
        PIPELINE-NAME-PREFIX: {pipeline_name}
        ROLE: {sm_role}
        TRACKING-SERVER-ARN: {mlflow_arn}
        """)

이제 다음 코드 셀에서 이러한 매개변수의 값을 위 셀에서 출력된 값으로 교체하세요.

이를 위해 `%%writefile codebuild-buildspec.yml`로 시작하는 다음 코드 셀에서 `kwargs` 매개변수를 찾으세요:


--kwargs "{ \
   \"input_s3_url\":\"<INPUT-S3-URL>\", \
   \"feature_group_name\":\"<FEATURE-GROUP-NAME>\", \
   \"model_package_group_name\":\"<MODEL-PACKAGE-GROUP-NAME>\", \
   \"pipeline_name_prefix\":\"<PIPELINE-BASE-NAME>\", \
   \"role\":\"<SAGEMAKER-EXECUTION-ROLE-ARN>\", \
   \"tracking_server_arn\":\"<TRACKING-SERVER-ARN>\", \
   \"region\":\"${AWS_REGION}\", \
   \"sagemaker_project_name\":\"${SAGEMAKER_PROJECT_NAME}\", \
   \"sagemaker_project_id\":\"${SAGEMAKER_PROJECT_ID}\", \
   \"bucket_name\":\"${ARTIFACT_BUCKET}\" \
       }"

그리고 `input_s3_url` 또는 `feature_group_name`, `model_package_group_name`, `pipeline_name_prefix`, `role`, `tracking_server_arn` 매개변수의 값을 이전 셀에서 출력된 값으로 교체하세요.

사용하려는 데이터셋 입력 방법에 따라 `input_s3_url` 또는 `feature_group_name` 중 하나만 교체하면 됩니다 - S3의 원시 입력 데이터셋 또는 피처 스토어의 처리된 피처셋. 이전 노트북에서 생성한 경우에만 피처 스토어를 사용할 수 있습니다.

<div class="alert alert-info">사용하지 않는 매개변수 줄을 셀 코드에서 삭제하세요: <code>input_s3_url</code> 또는 <code>feature_group_name</code>.</div>

![](img/codebuild-buildspec-edit.png)

매개변수 값을 교체한 후 셀을 실행하여 빌드 스펙 파일을 작성하세요.

In [ ]:
%%writefile codebuild-buildspec.yml

version: 0.2

phases:
  install:
    runtime-versions:
      python: 3.10
    commands:
      - pip install --upgrade --force-reinstall . "awscli>1.20.30"
      - pip install --upgrade mlflow==2.16.1 sagemaker-mlflow s3fs xgboost
    
  build:
    commands:
      - export SAGEMAKER_USER_CONFIG_OVERRIDE="./config.yaml"
      - export PYTHONUNBUFFERED=TRUE
      - export SAGEMAKER_PROJECT_NAME_ID="${SAGEMAKER_PROJECT_NAME}-${SAGEMAKER_PROJECT_ID}"
      - |
        run-pipeline --module-name pipelines.fromideatoprod.pipeline \
          --role-arn $SAGEMAKER_PIPELINE_ROLE_ARN \
          --tags "[{\"Key\":\"sagemaker:project-name\",\"Value\":\"${SAGEMAKER_PROJECT_NAME}\"}, {\"Key\":\"sagemaker:project-id\", \"Value\":\"${SAGEMAKER_PROJECT_ID}\"}]" \
          --kwargs "{ \
                \"input_s3_url\":\"s3://sagemaker-us-west-2-583558296381/from-idea-to-prod/xgboost/input/bank-additional-full.csv\", \
                \"feature_group_name\":\"from-idea-to-prod-17-09-05-47\", \
                \"model_package_group_name\":\"mlops-12-17-09-36-41-p-axctwyiytb2m\",\
                \"pipeline_name_prefix\":\"from-idea-to-prod-pipeline-17-08-50-14\",\
                \"role\":\"arn:aws:iam::583558296381:role/service-role/AmazonSageMaker-ExecutionRole-20201124T153725\",\
                \"tracking_server_arn\":\"arn:aws:sagemaker:us-west-2:583558296381:mlflow-tracking-server/mlflow-d-jvvzeggri2oe-04-14-04-38\", \
                \"region\":\"${AWS_REGION}\", \
                \"sagemaker_project_name\":\"${SAGEMAKER_PROJECT_NAME}\",\
                \"sagemaker_project_id\":\"${SAGEMAKER_PROJECT_ID}\",\
                \"bucket_name\":\"${ARTIFACT_BUCKET}\"\
                    }"
      - echo "Create/update of the SageMaker Pipeline and a pipeline execution completed."

워크샵 폴더에서 `codebuild-buildspec.yml` 파일을 프로젝트의 코드 저장소 폴더로 복사합니다:

In [ ]:
!cp ~/{workshop_folder}/codebuild-buildspec.yml ~/{project_path}/codebuild-buildspec.yml

요약하면, 빌드 스펙 파일에서 다음 세 가지 변경을 수행했습니다:
1. `run-pipeline` `--module-name` 매개변수 값을 `pipelines.abalone.pipeline`에서 새 경로 `pipelines.fromideatoprod.pipeline`로 수정
2. `get_pipeline()` 함수의 기본 매개변수 값을 사용하기 위해 `kwargs` 목록에서 일부 매개변수 제거
3. `kwargs` 매개변수 목록에 파이프라인을 위한 추가 매개변수 추가

### 4. `setup.py` 파일 생성
마지막으로 프로젝트의 코드 저장소 폴더에 `setup.py` 파일을 제공해야 합니다.

In [ ]:
%%writefile setup.py
import os
import setuptools


about = {}
here = os.path.abspath(os.path.dirname(__file__))
with open(os.path.join(here, "pipelines", "__version__.py")) as f:
    exec(f.read(), about)


with open("README.md", "r") as f:
    readme = f.read()


required_packages = ["sagemaker==2.255.0"]
extras = {
    "test": [
        "black",
        "coverage",
        "flake8",
        "mock",
        "pydocstyle",
        "pytest",
        "pytest-cov",
        "sagemaker",
        "tox",
    ]
}
setuptools.setup(
    name=about["__title__"],
    description=about["__description__"],
    version=about["__version__"],
    author=about["__author__"],
    author_email=["__author_email__"],
    long_description=readme,
    long_description_content_type="text/markdown",
    url=about["__url__"],
    license=about["__license__"],
    packages=setuptools.find_packages(),
    include_package_data=True,
    python_requires=">=3.6",
    install_requires=required_packages,
    extras_require=extras,
    entry_points={
        "console_scripts": [
            "get-pipeline-definition=pipelines.get_pipeline_definition:main",
            "run-pipeline=pipelines.run_pipeline:main",
        ]
    },
    classifiers=[
        "Development Status :: 3 - Alpha",
        "Intended Audience :: Developers",
        "Natural Language :: English",
        "Programming Language :: Python",
        "Programming Language :: Python :: 3",
        "Programming Language :: Python :: 3.6",
        "Programming Language :: Python :: 3.7",
        "Programming Language :: Python :: 3.8",
    ],
)

In [ ]:
!cp ~/{workshop_folder}/setup.py ~/{project_path}/setup.py

---

## 모델 빌드 파이프라인을 위한 CI/CD 실행
모델 빌드 파이프라인을 위한 CI/CD를 시작하려면 변경된 코드를 프로젝트 GitHub 저장소에 푸시해야 합니다.

<div class="alert alert-info">git 명령을 실행할 때 JupyterLab 터미널에서 저장소 코드가 포함된 폴더에 있는지 확인하세요. 폴더 이름은 <code>[project-name]/sagemaker-[project-id]-modelbuild</code>와 같습니다.</div>

JupyterLab 메뉴 **File** > **New** > **Terminal**을 통해 시스템 터미널 창을 열고 다음 코드 셀에서 생성된 명령을 입력하세요.
`user.email`과 `user.name`을 유지하거나 본인의 데이터로 교체하세요.

In [ ]:
cmd = f'''
cd ~/{project_path}

git config --global user.email "you@example.com"
git config --global user.name "Your Name"
  
git add -A
git commit -am "customize project"
git push
'''

copy_output(cmd)

코드 변경 사항을 푸시한 후, 프로젝트는 SageMaker 모델 빌드 파이프라인을 구성, 업서트 및 실행하는 CodePipeline 파이프라인 실행을 시작합니다. 이 새로운 파이프라인 실행은 SageMaker 모델 레지스트리의 모델 패키지 그룹에 새 모델 버전을 생성합니다.

Studio **Pipelines** 위젯에서 파이프라인 실행을 추적할 수 있습니다.

파이프라인 실행이 완료될 때까지 기다리세요. 실행 완료까지 약 10분이 소요됩니다.

파이프라인 실행을 보려면 아래 코드 셀에서 구성된 링크를 클릭하세요. CodeBuild가 파이프라인을 빌드 업서트하고 실행을 시작하는 데 약 1분이 소요됩니다. 시작된 실행을 보려면 Studio UI 페이지를 새로고침하세요.

In [ ]:
# Show the pipeline execution link
display(
    HTML('<b>See <a target="top" href="https://studio-{}.studio.{}.sagemaker.aws/pipelines/{}/executions/">the pipeline executions</a> in the Studio UI</b>'.format(
            domain_id, region, p.describe()['PipelineName']))
)

## 모델 레지스트리에서 모델 패키지 보기

이 새로운 파이프라인은 SageMaker 모델 레지스트리에 `<project_name>-<project-id>`라는 이름의 새 모델 패키지 그룹을 생성합니다.

Studio UI에서 모델 패키지 버전을 보려면 아래 코드 셀에서 구성된 링크를 클릭하세요.
<div class="alert alert-info">모델 패키지에서 등록된 모델 버전을 보려면 파이프라인 실행이 완료될 때까지 기다려야 합니다.</div>

열리는 모델 버전 탭에서 활동, [모델 버전 세부 정보](https://docs.aws.amazon.com/sagemaker/latest/dg/model-registry-details.html) 및 [데이터 계보](https://docs.aws.amazon.com/sagemaker/latest/dg/lineage-tracking.html)를 탐색할 수 있습니다.

실제 프로젝트에서는 [모델 품질 메트릭](https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor-model-quality-metrics.html), [설명 가능성](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-model-explainability.html) 및 [편향](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-measure-post-training-bias.html) 보고서, 로드 테스트 데이터, [추론 추천기](https://docs.aws.amazon.com/sagemaker/latest/dg/inference-recommender.html)와 같은 다양한 모델 속성과 추가 모델 버전 메타데이터를 추가합니다.

In [ ]:
# Show the model package link
display(
    HTML('<b>See <a target="top" href="https://studio-{}.studio.{}.sagemaker.aws/models/registered-models/{}-{}/versions">the model package versions</a> in the Studio UI</b>'.format(
            domain_id, region, project_name, project_id))
)

## 요약
이 노트북에서는 다음 기능을 가진 CI/CD 파이프라인을 구현합니다:
- 모델 빌드 ML 파이프라인이 GitHub 저장소의 소스 제어 하에 있습니다.
- 코드 저장소에 푸시할 때마다 ML 파이프라인을 구성, 업서트 및 실행하는 새로운 CodePipeline 파이프라인이 시작됩니다.
- 전체 엔드투엔드 모델 개발 프로세스가 이제 자동화되었습니다.
- SageMaker 프로젝트는 관련 ML 파이프라인, 저장소, 모델, 실험 및 추론 엔드포인트에 대한 메타데이터를 가진 Studio의 논리적 구조입니다.

---

## 5단계 계속하기
다음 노트북 [05-deploy](05-deploy.ipynb)에서 MLOps 프로젝트의 두 번째 부분인 모델 배포 파이프라인을 테스트합니다.

## 실제 프로젝트를 위한 추가 개발 아이디어
- SageMaker에서 제공하는 [Jenkins를 사용한 타사 Git 저장소와 함께하는 모델 빌드, 훈련 및 배포를 위한 MLOps 템플릿](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects-templates-sm.html#sagemaker-projects-templates-git-jenkins)을 사용할 수 있습니다.
- 특정 프로젝트 요구 사항을 충족하기 위해 [커스텀 SageMaker 프로젝트 템플릿](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects-templates-custom.html)을 생성하세요.

## 추가 리소스
- [SageMaker Immersion Day의 Amazon SageMaker Pipelines 실습](https://catalog.us-east-1.prod.workshops.aws/workshops/63069e26-921c-4ce1-9cc7-dd882ff62575/en-US/lab6)
- [Amazon SageMaker 프로젝트와 함께 모듈식 아키텍처를 사용하여 머신러닝 개발 향상하기](https://aws.amazon.com/blogs/machine-learning/enhance-your-machine-learning-development-by-using-a-modular-architecture-with-amazon-sagemaker-projects/)
- [MLOps 자동화 심화 탐구](https://www.youtube.com/watch?v=3_cHnk9VSfQ)
- [SageMaker MLOps 프로젝트 연습](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects-walkthrough.html)
- [커스텀 프로젝트 템플릿 예제가 있는 `aws-samples` GitHub 저장소](https://github.com/aws-samples/sagemaker-custom-project-templates)

# kernel 정지

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>